# Generate Permutation Importance (i.e., rank) from SHAP files

In [1]:
import shap
import numpy as np
import pickle
import glob
import pandas as pd
import subprocess
import csv

In [2]:
import sys
sys.path.append('/explore/nobackup/people/gtamkin/dev/AGB/mpf-model-factories/MultiPathFusion')
sys.path.append('/explore/nobackup/people/gtamkin/dev/AGB/monte-carlo/monte-carlo')
sys.path.append('/panfs/ccds02/app/modules/anaconda/platform/x86_64/rhel/8.6/3-2022.05/envs/ilab-tensorflow/bin')

In [3]:
def format_band_set(subset):
    indexListIntStr = ''
    count = 0
    for band in subset:
        if (count == 0):
            indexListIntStr = indexListIntStr + str(band)
        else:
            indexListIntStr = indexListIntStr + ', ' + str(band)
        count = count + 1
    return indexListIntStr


In [4]:
# cmd = "/panfs/ccds02/app/modules/anaconda/platform/x86_64/rhel/8.6/3-2022.05/envs/ilab-tensorflow/bin/gdalinfo /explore/nobackup/projects/ilab/data/AGB/test/mlruns/aggregate/MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background.tif| grep Band | wc -l"
# # returned_value = subprocess.call(cmd, shell=True)  # returns the exit code in unix
# # print('returned value:', returned_value)
# import os
# returned_value = os.system(cmd)  # returns the exit code in unix
# print('returned value:', returned_value)


In [5]:
Bands = !/panfs/ccds02/app/modules/anaconda/platform/x86_64/rhel/8.6/3-2022.05/envs/ilab-tensorflow/bin/gdalinfo /explore/nobackup/projects/ilab/data/AGB/test/mlruns/aggregate/MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background.tif| grep Band | wc -l
print('Bands: ', Bands)

Bands:  ['447']


## Read batch of shap files for experiment, which contains run-time artifacts (e.g., SHAP values)

In [16]:
# /app/jupyter/ilab/tensorflow-kernel/bin/python3.8 /panfs/ccds02/nobackup/people/gtamkin/dev/AGB/monte-carlo/monte-carlo/mc/view/MpfSimulationMultiprocess.py \
#    --config ./tests/exp_aggregate_hyperspectral_callbacks_noMinMax.json -o /explore/nobackup/projects/ilab/data/AGB/test/mlruns/aggregate/20241020-gpu001-callbacks-divideby10k_noMinMax \
#    --bandListFile /explore/nobackup/projects/ilab/data/AGB/test/mlruns/random_sets/random_sets_20240111_shrink_False_num_samples_1088_bandLen_447 \
#    -hf /explore/nobackup/projects/ilab/data/AGB/test/mlruns/aggregate/MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif \
#    -t 1200 

# initialize list of lists
headers = ['Top10Index','Top10Rank','Scaling','#Bands','#Trials','Band/Trial','MaxEpochs','hh:mm:ss.ms','Host(s)','CPUs/Host','Concurrency','ResultDir','ID','Description','HF', 'Callbacks']
root_data_path = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/aggregate'
site = 'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0'

permutation_results = [
     ['NoTop10Index', 'Top10Rank', 'DivBy10K','447', '1088', '10', '50', '10:13:32.980461','gpu001','40','Pool', '202410130-gpu004-40cpus-callbacks-divideby10k_noMinMax','225919640119481430', 
      '### gpu004: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized ("scale_data_method": "None")',
      'hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],
     ['NoTop10Index', 'Top10Rank', 'DivBy10K','447', '1089', '10', '50', '9:44:57.880395','gpu001','40','Pool', '202410125-gpu001-callbacks-divideby10k_noMinMax','393911649901623133', 
      '### gpu001: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized ("scale_data_method": "None")',
      'hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],
     ['NoTop10Index', 'Top10Rank', 'DivBy10K','447', '1088', '10', '50', '9:57:19.194150','gpu001','40','Pool', '202410121b-gpu001-callbacks-divideby10k_noMinMax','971589025605583160', 
      '### gpu001: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized ("scale_data_method": "None")',
      'hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],
     ['NoTop10Index', 'Top10Rank', 'DivBy10K','447', '1088', '10', '50', '9:38:07.808645','gpu001','40','Pool', '202410121-gpu001-callbacks-divideby10k_noMinMax','916501809805875784', 
      '### gpu001: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized ("scale_data_method": "None")',
      'hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],
     ['NoTop10Index', 'Top10Rank', 'DivBy10K','447', '1088', '10', '50', '9:29:50.877797','gpu001','40','Pool', '202410120-gpu001-callbacks-divideby10k_noMinMax','871912588490665214', 
      '### gpu001: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized ("scale_data_method": "None")',
      'hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],
     ['NoTop10Index', 'Top10Rank', 'DivBy10K','447', '1088', '10', '50', 'NoMinutes','ilab[201-212,211]','10','pdsh', '202410120-callbacks-divideby10k_noMinMax','648862663793551468', 
      '### CPU pdsh[201-212,211]: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized',
      'hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],
    ['NoTop10Index', 'Top10Rank', 'DivBy10K','447', '1088', '10', '50', '4:24:16.993903','ilab[201-212]','10','pdsh', '20241019-callbacks-divideby10k_noMinMax','964586160942144948', 
      '### CPU pdsh[201-212]: 50 Epochs (w/ EarlyStopping callbacks) after scaling NEON by 10,000 - NO bands normalized',
      'hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],
    ['NoTop10Index', 'Top10Rank','minmax','447', '1088', '10', '50', '9:36:55.985009','gpu003','40','Pool', '20240118-callbacks-gpu','504373152637067057', 
      'gpu003: Epochs = 50 (w/ Early callbacks) and explainer.shap_values(X.iloc[0:50, :], nsamples=150) - All bands normalized ("scale_data_method": "minmax")',
      'hyperspectral-aggregate.separate-background.tif','EarlyWarning'],
    ['NoTop10Index', 'Top10Rank','minmax','447', '1088', '10', '50', '9:34:11.607509','gpu003','40','Pool', '20240117-callbacks-gpu','660588493840123655', 
      'gpu003: Epochs = 50 (w/ Early callbacks) and explainer.shap_values(X.iloc[0:50, :], nsamples=150) - All bands normalized ("scale_data_method": "minmax")',
      'hyperspectral-aggregate.separate-background.tif','EarlyWarning'],
    ['NoTop10Index', 'Top10Rank', 'minmax','447', '1088', '10', '3', '9:34:11.607509','gpu003','40','Pool', '20240113_gpu','342218863809278249', 
      'GPU: Epochs = 3 and explainer.shap_values(X.iloc[0:50, :], nsamples=150) - All bands normalized ("scale_data_method": "minmax")',
      'hyperspectral-aggregate.separate-background.tif','EarlyWarning'],
    ['NoTop10Index', 'Top10Rank','minmax','447', '1088', '10', '2', '9:34:11.607509','gpu003','40','Pool', '20240112','789617681324607598', 
      'CPU & GPU: Epochs = 2 and explainer.shap_values(X.iloc[0:50, :], nsamples=500) - All bands normalized ("scale_data_method": "minmax")',
      'hyperspectral-aggregate.separate-background.tif','EarlyWarning'],
]
 
# Create the pandas DataFrame
df = pd.DataFrame(permutation_results, columns=headers)

# for index, row in df.iterrows():
#     print(row['Scaling'], row['Results'])

# # select two columns
# print(df[['Scaling', 'Results']])

# print(df.at[1,'Results'])

# for index, row in df.iterrows():
#     print(index,'index: ', df.at[index,'Results'])
    
# print dataframe.
df

,Top10Index,Top10Rank,Scaling,#Bands,#Trials,Band/Trial,MaxEpochs,hh:mm:ss.ms,Host(s),CPUs/Host,Concurrency,ResultDir,ID,Description,HF,Callbacks
0,NoTop10Index,Top10Rank,DivBy10K,447,1088,10,50,10:13:32.980461,gpu001,40,Pool,202410130-gpu004-40cpus-callbacks-divideby10k_...,225919640119481430,### gpu004: 50 Epochs (w/ EarlyStopping callba...,hyperspectral-aggregate.separate-background-sc...,EarlyWarning
1,NoTop10Index,Top10Rank,DivBy10K,447,1089,10,50,9:44:57.880395,gpu001,40,Pool,202410125-gpu001-callbacks-divideby10k_noMinMax,393911649901623133,### gpu001: 50 Epochs (w/ EarlyStopping callba...,hyperspectral-aggregate.separate-background-sc...,EarlyWarning
2,NoTop10Index,Top10Rank,DivBy10K,447,1088,10,50,9:57:19.194150,gpu001,40,Pool,202410121b-gpu001-callbacks-divideby10k_noMinMax,971589025605583160,### gpu001: 50 Epochs (w/ EarlyStopping callba...,hyperspectral-aggregate.separate-background-sc...,EarlyWarning
3,NoTop10Index,Top10Rank,DivBy10K,447,1088,10,50,9:38:07.808645,gpu001,40,Pool,202410121-gpu001-callbacks-divideby10k_noMinMax,916501809805875784,### gpu001: 50 Epochs (w/ EarlyStopping callba...,hyperspectral-aggregate.separate-background-sc...,EarlyWarning
4,NoTop10Index,Top10Rank,DivBy10K,447,1088,10,50,9:29:50.877797,gpu001,40,Pool,202410120-gpu001-callbacks-divideby10k_noMinMax,871912588490665214,### gpu001: 50 Epochs (w/ EarlyStopping callba...,hyperspectral-aggregate.separate-background-sc...,EarlyWarning
5,NoTop10Index,Top10Rank,DivBy10K,447,1088,10,50,NoMinutes,"ilab[201-212,211]",10,pdsh,202410120-callbacks-divideby10k_noMinMax,648862663793551468,"### CPU pdsh[201-212,211]: 50 Epochs (w/ Early...",hyperspectral-aggregate.separate-background-sc...,EarlyWarning
6,NoTop10Index,Top10Rank,DivBy10K,447,1088,10,50,4:24:16.993903,ilab[201-212],10,pdsh,20241019-callbacks-divideby10k_noMinMax,964586160942144948,### CPU pdsh[201-212]: 50 Epochs (w/ EarlyStop...,hyperspectral-aggregate.separate-background-sc...,EarlyWarning
7,NoTop10Index,Top10Rank,minmax,447,1088,10,50,9:36:55.985009,gpu003,40,Pool,20240118-callbacks-gpu,504373152637067057,gpu003: Epochs = 50 (w/ Early callbacks) and e...,hyperspectral-aggregate.separate-background.tif,EarlyWarning
8,NoTop10Index,Top10Rank,minmax,447,1088,10,50,9:34:11.607509,gpu003,40,Pool,20240117-callbacks-gpu,660588493840123655,gpu003: Epochs = 50 (w/ Early callbacks) and e...,hyperspectral-aggregate.separate-background.tif,EarlyWarning
9,NoTop10Index,Top10Rank,minmax,447,1088,10,3,9:34:11.607509,gpu003,40,Pool,20240113_gpu,342218863809278249,GPU: Epochs = 3 and explainer.shap_values(X.il...,hyperspectral-aggregate.separate-background.tif,EarlyWarning


# Read from CSV

In [17]:
hyperspectral_aggregate_bandnames = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/aggregate/MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate-bandnames.csv'
only_row = None
with open(hyperspectral_aggregate_bandnames) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        only_row = row
        print(row[0])
        print(row[446])
print(only_row)

ARI
2511.7
['ARI', 'CAI', 'CRI550', 'CRI700', 'EVI', 'EVI2', 'fPAR', 'LAI', 'MCTI', 'MSI', 'NDII', 'NDLI', 'NDNI', 'NDVI', 'NDWI', 'NIRv', 'PRIn', 'PRIw', 'SAVI', 'WBI', 'Albedo', '383.072', '388.0806', '393.0891', '398.0977', '403.1062', '408.1147', '413.1233', '418.1318', '423.1403', '428.1489', '433.1574', '438.1659', '443.1745', '448.183', '453.1916', '458.2001', '463.2086', '468.2172', '473.2257', '478.2342', '483.2428', '488.2513', '493.2598', '498.2684', '503.2769', '508.2855', '513.294', '518.3025', '523.3111', '528.3196', '533.3281', '538.3367', '543.3452', '548.3537', '553.3623', '558.3708', '563.3794', '568.3879', '573.3964', '578.405', '583.4135', '588.422', '593.4306', '598.4391', '603.4476', '608.4562', '613.4647', '618.4733', '623.4818', '628.4903', '633.4989', '638.5074', '643.5159', '648.5245', '653.533', '658.5415', '663.5501', '668.5586', '673.5672', '678.5757', '683.5842', '688.5928', '693.6013', '698.6098', '703.6184', '708.6269', '713.6354', '718.644', '723.6525',

# Loop through result files and create the ranking

In [18]:
#. Loop through result files and create the ranking
for r_index, row in df.iterrows():
    shap_dir_prefix = root_data_path + '/' + df.at[r_index,'ResultDir'] + '/' + df.at[r_index,'ID'] 
    shap_dir = shap_dir_prefix + '/PERMUTATION_IMPORTANCE_VALUES/'
    print('\n ======== index[', r_index, '] ==========')
    print(r_index,'index: ', shap_dir)

    # Returns a list of names in list files. 
#    print("Using glob.glob()") 
    shap_files = glob.glob(shap_dir + '**/*.shap_values0to50',  
                       recursive = True) 
#    print('shap_files[',r_index,']: ', shap_files[r_index])  

    # set to max number of hyperspectral indices in experiment
    bandLen = int(df.at[r_index,'#Bands'])
#    print(bandLen)
    bandOccurenceArr = np.zeros(bandLen).astype(int)
    bandAbsSumArr = np.zeros(bandLen)

    first_time = True
    index = 0
    # Reload config
    start = left = '[['
    end = right = ']]'

    for shap_vals_path in shap_files:

        index = index+1
        num_shap_files = len(shap_files)
        bandListStr = shap_vals_path[shap_vals_path.index(left)+len(left):shap_vals_path.index(right)]
        bandList = [e for e in bandListStr.split(',')]
        for bandNum in bandList:
            bandOccurenceArr[int(bandNum)-1] = (bandOccurenceArr[int(bandNum)-1]) + 1

        # Reload shap values
        shap_values0to50 = pickle.load(open(shap_vals_path, "rb"))
        num_bins = len(shap_values0to50)
        num_rows = len(shap_values0to50[0])
        num_bands = num_cols = len(shap_values0to50[0][0])
#        if (index == 1): 
#            print('\nProcessing shap file: index ', index,' out of ', num_shap_files, ':', os.path.basename(shap_vals_path))
#            print('num_bins (', num_bins, ') num_rows (', num_rows, ') num_bands = num_cols(', num_bands, ') bandList(', bandList, ')')
#            print('bandOccurenceArr after first mods(', bandOccurenceArr, ')')


        #  Walk across bins [0,4]
        for _bin in range(0, num_bins):
#            if ((_bin == 0) and (index == 1)): print('len(shap_values0to50[',_bin,'])', len(shap_values0to50[_bin]))

            # get absolute value of all values in matrix
            shap_values0to50_bin_abs = np.abs(shap_values0to50[_bin])
#            if ((_bin == 0) and (index == 1)): print('shap_values0to50_bin_abs.shape  : ', shap_values0to50_bin_abs.shape)

            # convert numpy matrix to dataframe and print sum of absolute values in each column
            df_shap_values0to50_bin_abs = pd.DataFrame(data=shap_values0to50_bin_abs)
#            if ((_bin == 0) and (index == 1)): print('df_shap_values0to50_bin_abs.shape  : ', df_shap_values0to50_bin_abs.shape)

            # create a row (numpy array) where each column is the sum of the absolute values in that vertical column
            shap_values_abs_sum_axis0 = np.sum(df_shap_values0to50_bin_abs, axis=0)
#            if ((_bin == 0) and (index == 1)): print('shap_values_abs_sum_axis0.shape  : ', shap_values_abs_sum_axis0.shape)

            # sort the row from low to high to determine the rank of the columns (e.g., features)
            shap_values_abs_sum_argsort = np.argsort(shap_values_abs_sum_axis0)
 #           if ((_bin == 0) and (index == 1)): print('L->H (index == 1):\n', shap_values_abs_sum_argsort)


 #           break;
            # loop thru each column and update the corresponding statistics in the global row
            for _col in range(0, num_cols):
                bandNum = int(bandList[_col])
#                if ((_bin == 0) and (index == 1)): print('bandNum[', bandNum,'] column[',_col,'] band[',bandList[_col].strip(),'] sum(abs)', df_shap_values0to50_bin_abs[_col].sum())

                # add the absolute value of the current shape_value to the existing value in the cell that is indexed by the band number
                # this allows us to keep a running total shap values per feature for downselection later....
                bandAbsSumArr[bandNum-1] = (bandAbsSumArr[bandNum-1] + df_shap_values0to50_bin_abs[_col].sum())

    ascending_indices = np.argsort(bandAbsSumArr)[:10]
    # print('Final.min:', bandAbsSumArr.min(), 'Final.max:', bandAbsSumArr.max())
    # print('Final L->H:', ascending_indices)
    # print('Final L->H[0]:', ascending_indices[0])
    minIndex = ascending_indices[0]
    # print('bandAbsSumArr[',minIndex,']', bandAbsSumArr[minIndex])


    descending_indices = np.argsort(bandAbsSumArr)[::-1][:10]
    # print('\nFinal H->L:', descending_indices)
    # print('Final H->L[0]:', descending_indices[0])
    maxIndex = descending_indices[0]
    # print('bandAbsSumArr[',maxIndex,']', bandAbsSumArr[maxIndex])

    with np.printoptions(precision=5, suppress=True):
#        print('Final bandOccurenceArr (', bandOccurenceArr, ')')
#        print('Final bandAbsSumArr (',bandAbsSumArr)
        print('Final:', bandAbsSumArr[:10])
        print('Final.min:', bandAbsSumArr.min(), 'Final.max:', bandAbsSumArr.max())
        print('Final L->H:', np.argsort(bandAbsSumArr)[:10])
        row['Top10Index'] = np.argsort(bandAbsSumArr)[::-1][:10]
        
        indexList = np.argsort(bandAbsSumArr)[::-1][:10]
        bandList = []
        for index in range(0, 10):
        #for (index < 10):
            bandList.append(str(only_row[indexList[index]]))
            print(indexList[index], only_row[indexList[index]])        
            index = index + 1
        row['Top10Rank'] = bandList
        
        print('Final H->L:', row['Top10Index'] )
        print('Final max:', row['Top10Index'][0])
#    print('bandAvgArr: ',bandAvgArr)



 ======== index[ 0 ] ==========
0 index:  /explore/nobackup/projects/ilab/data/AGB/test/mlruns/aggregate/202410130-gpu004-40cpus-callbacks-divideby10k_noMinMax/225919640119481430/PERMUTATION_IMPORTANCE_VALUES/
Final: [ 0.00938  0.00095  0.0085   0.02303  0.31805  0.26134  4.8651  10.69957
  1.78652  0.10135]
Final.min: 0.0009518336286996796 Final.max: 52.72887673005679
Final L->H: [ 1 26 28 24 25 23 36 27 31 30]
15 NIRv
7 LAI
6 fPAR
8 MCTI
221 1384.7793
217 1364.7452
216 1359.7366
218 1369.7537
223 1394.7964
323 1895.65
Final H->L: [ 15   7   6   8 221 217 216 218 223 323]
Final max: 15

 ======== index[ 1 ] ==========
1 index:  /explore/nobackup/projects/ilab/data/AGB/test/mlruns/aggregate/202410125-gpu001-callbacks-divideby10k_noMinMax/393911649901623133/PERMUTATION_IMPORTANCE_VALUES/
Final: [ 0.01156  0.00095  0.01033  0.026    0.26257  0.31856 12.74993  8.04698
  2.02905  0.11245]
Final.min: 0.0009535379857036018 Final.max: 45.09472327782323
Final L->H: [ 1 36 26 28 30 25 41 31 24

In [19]:
df

,Top10Index,Top10Rank,Scaling,#Bands,#Trials,Band/Trial,MaxEpochs,hh:mm:ss.ms,Host(s),CPUs/Host,Concurrency,ResultDir,ID,Description,HF,Callbacks
0,"[15, 7, 6, 8, 221, 217, 216, 218, 223, 323]","[NIRv, LAI, fPAR, MCTI, 1384.7793, 1364.7452, ...",DivBy10K,447,1088,10,50,10:13:32.980461,gpu001,40,Pool,202410130-gpu004-40cpus-callbacks-divideby10k_...,225919640119481430,### gpu004: 50 Epochs (w/ EarlyStopping callba...,hyperspectral-aggregate.separate-background-sc...,EarlyWarning
1,"[15, 6, 7, 8, 218, 217, 216, 221, 223, 323]","[NIRv, fPAR, LAI, MCTI, 1369.7537, 1364.7452, ...",DivBy10K,447,1089,10,50,9:44:57.880395,gpu001,40,Pool,202410125-gpu001-callbacks-divideby10k_noMinMax,393911649901623133,### gpu001: 50 Epochs (w/ EarlyStopping callba...,hyperspectral-aggregate.separate-background-sc...,EarlyWarning
2,"[15, 7, 8, 218, 216, 221, 217, 223, 323, 219]","[NIRv, LAI, MCTI, 1369.7537, 1359.7366, 1384.7...",DivBy10K,447,1088,10,50,9:57:19.194150,gpu001,40,Pool,202410121b-gpu001-callbacks-divideby10k_noMinMax,971589025605583160,### gpu001: 50 Epochs (w/ EarlyStopping callba...,hyperspectral-aggregate.separate-background-sc...,EarlyWarning
3,"[15, 7, 6, 8, 218, 216, 221, 217, 223, 323]","[NIRv, LAI, fPAR, MCTI, 1369.7537, 1359.7366, ...",DivBy10K,447,1088,10,50,9:38:07.808645,gpu001,40,Pool,202410121-gpu001-callbacks-divideby10k_noMinMax,916501809805875784,### gpu001: 50 Epochs (w/ EarlyStopping callba...,hyperspectral-aggregate.separate-background-sc...,EarlyWarning
4,"[15, 7, 6, 8, 216, 217, 218, 221, 223, 326]","[NIRv, LAI, fPAR, MCTI, 1359.7366, 1364.7452, ...",DivBy10K,447,1088,10,50,9:29:50.877797,gpu001,40,Pool,202410120-gpu001-callbacks-divideby10k_noMinMax,871912588490665214,### gpu001: 50 Epochs (w/ EarlyStopping callba...,hyperspectral-aggregate.separate-background-sc...,EarlyWarning
5,"[15, 6, 8, 7, 221, 218, 217, 323, 223, 216]","[NIRv, fPAR, MCTI, LAI, 1384.7793, 1369.7537, ...",DivBy10K,447,1088,10,50,NoMinutes,"ilab[201-212,211]",10,pdsh,202410120-callbacks-divideby10k_noMinMax,648862663793551468,"### CPU pdsh[201-212,211]: 50 Epochs (w/ Early...",hyperspectral-aggregate.separate-background-sc...,EarlyWarning
6,"[15, 7, 8, 218, 216, 217, 221, 223, 323, 219]","[NIRv, LAI, MCTI, 1369.7537, 1359.7366, 1364.7...",DivBy10K,447,1088,10,50,4:24:16.993903,ilab[201-212],10,pdsh,20241019-callbacks-divideby10k_noMinMax,964586160942144948,### CPU pdsh[201-212]: 50 Epochs (w/ EarlyStop...,hyperspectral-aggregate.separate-background-sc...,EarlyWarning
7,"[310, 218, 323, 327, 217, 216, 223, 221, 322, ...","[1830.5391, 1369.7537, 1895.65, 1915.6842, 136...",minmax,447,1088,10,50,9:36:55.985009,gpu003,40,Pool,20240118-callbacks-gpu,504373152637067057,gpu003: Epochs = 50 (w/ Early callbacks) and e...,hyperspectral-aggregate.separate-background.tif,EarlyWarning
8,"[323, 218, 310, 327, 216, 221, 217, 223, 322, ...","[1895.65, 1369.7537, 1830.5391, 1915.6842, 135...",minmax,447,1088,10,50,9:34:11.607509,gpu003,40,Pool,20240117-callbacks-gpu,660588493840123655,gpu003: Epochs = 50 (w/ Early callbacks) and e...,hyperspectral-aggregate.separate-background.tif,EarlyWarning
9,"[218, 310, 221, 216, 217, 323, 327, 223, 161, ...","[1369.7537, 1830.5391, 1384.7793, 1359.7366, 1...",minmax,447,1088,10,3,9:34:11.607509,gpu003,40,Pool,20240113_gpu,342218863809278249,GPU: Epochs = 3 and explainer.shap_values(X.il...,hyperspectral-aggregate.separate-background.tif,EarlyWarning


In [20]:
print('Summary: [Job Index]-[Top 10 Rank]-[Top 10 Indices]-[Scaling]')
for index, row in df.iterrows():
    print(index, '-',row['Top10Rank'], '-', row['Top10Index'], '-', row['Scaling'])

Summary: [Job Index]-[Top 10 Rank]-[Top 10 Indices]-[Scaling]
0 - ['NIRv', 'LAI', 'fPAR', 'MCTI', '1384.7793', '1364.7452', '1359.7366', '1369.7537', '1394.7964', '1895.65'] - [ 15   7   6   8 221 217 216 218 223 323] - DivBy10K
1 - ['NIRv', 'fPAR', 'LAI', 'MCTI', '1369.7537', '1364.7452', '1359.7366', '1384.7793', '1394.7964', '1895.65'] - [ 15   6   7   8 218 217 216 221 223 323] - DivBy10K
2 - ['NIRv', 'LAI', 'MCTI', '1369.7537', '1359.7366', '1384.7793', '1364.7452', '1394.7964', '1895.65', '1374.7622'] - [ 15   7   8 218 216 221 217 223 323 219] - DivBy10K
3 - ['NIRv', 'LAI', 'fPAR', 'MCTI', '1369.7537', '1359.7366', '1384.7793', '1364.7452', '1394.7964', '1895.65'] - [ 15   7   6   8 218 216 221 217 223 323] - DivBy10K
4 - ['NIRv', 'LAI', 'fPAR', 'MCTI', '1359.7366', '1364.7452', '1369.7537', '1384.7793', '1394.7964', '1910.6757'] - [ 15   7   6   8 216 217 218 221 223 326] - DivBy10K
5 - ['NIRv', 'fPAR', 'MCTI', 'LAI', '1384.7793', '1369.7537', '1364.7452', '1895.65', '1394.7964

# DEMO 01/31/2024... Went great!

In [19]:
for index, row in df.iterrows():
    print(row['Top10Index'], row['ResultDir'])


for index, row in df.iterrows():
    print(row['Scaling'], row['ResultDir'])

# select two columns
print(df[['Scaling', 'ResultDir']])

print(df.at[1,'ResultDir'])

for index, row in df.iterrows():
    print(index,'index: ', df.at[index,'ResultDir'])

DivBy10K 202410125-gpu001-callbacks-divideby10k_noMinMax
DivBy10K 202410121b-gpu001-callbacks-divideby10k_noMinMax
DivBy10K 202410121-gpu001-callbacks-divideby10k_noMinMax
DivBy10K 202410120-gpu001-callbacks-divideby10k_noMinMax
DivBy10K 202410120-callbacks-divideby10k_noMinMax
DivBy10K 20241019-callbacks-divideby10k_noMinMax
minmax 20240118-callbacks-gpu
minmax 20240117-callbacks-gpu
minmax 20240113_gpu
minmax 20240112
    Scaling                                         ResultDir
0  DivBy10K   202410125-gpu001-callbacks-divideby10k_noMinMax
1  DivBy10K  202410121b-gpu001-callbacks-divideby10k_noMinMax
2  DivBy10K   202410121-gpu001-callbacks-divideby10k_noMinMax
3  DivBy10K   202410120-gpu001-callbacks-divideby10k_noMinMax
4  DivBy10K          202410120-callbacks-divideby10k_noMinMax
5  DivBy10K           20241019-callbacks-divideby10k_noMinMax
6    minmax                            20240118-callbacks-gpu
7    minmax                            20240117-callbacks-gpu
8    minmax       

In [28]:
indexList = np.argsort(bandAbsSumArr)[::-1][:10]
for index in range(0, 9):
#for (index < 10):
    index = index + 1
    print(indexList[index], only_row[indexList[index]])

218 1369.7537
161 1084.2671
216 1359.7366
221 1384.7793
166 1109.3098
144 999.122
118 868.9001
164 1099.2927
155 1054.216


In [ ]:
ascending_indices = np.argsort(bandAbsSumArr)[:10]
print('Final.min:', bandAbsSumArr.min(), 'Final.max:', bandAbsSumArr.max())
print('Final L->H:', ascending_indices)
print('Final L->H[0]:', ascending_indices[0])
minIndex = ascending_indices[0]
print('bandAbsSumArr[',minIndex,']', bandAbsSumArr[minIndex])


descending_indices = np.argsort(bandAbsSumArr)[::-1][:10]
print('\nFinal H->L:', descending_indices)
print('Final H->L[0]:', descending_indices[0])
maxIndex = descending_indices[0]
print('bandAbsSumArr[',maxIndex,']', bandAbsSumArr[maxIndex])


In [ ]:
arr = bandAbsSumArr
 
# get the indices that would sort the array in ascending order
ascending_indices = arr.argsort()  # [1 3 0 4 2 5]
 
# reverse the ascending indices to get descending indices
descending_indices = ascending_indices[::-1]  # [5 2 4 0 3 1]
 
# use the descending indices to get the elements in descending order
sorted_elements_descending = arr[descending_indices]  # [10 8 6 5 3 2]
 
print("Sorted Indices (Descending):", descending_indices)
print("Sorted Elements (Descending):", sorted_elements_descending)

In [ ]:
descending_indices[0:10]

In [ ]:
descending_indices_str = format_band_set(descending_indices[0:10])
descending_indices_str = '[' + descending_indices_str + ']'
descending_indices_str

In [ ]:
df

In [ ]:
# Create the pandas DataFrame

df = pd.DataFrame(permutation_results, columns=headers)

for index, row in df.iterrows():
    print(row['Scaling'], row['Results'])


In [ ]:
#========= ORIGINAL LOGIC BELOW =============

In [ ]:
shap_dir = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/aggregate/20241019-callbacks-divideby10k_noMinMax/964586160942144948/PERMUTATION_IMPORTANCE_VALUES/'
# Returns a list of names in list files. 
print("Using glob.glob()") 
shap_files = glob.glob(shap_dir + '**/*.shap_values0to50',  
                       recursive = True) 
# for file in cfg_files: 
#     print(file) 
print('shap_files[0]: ', shap_files[0], '\nnum shap files: ', len(shap_files))  


In [ ]:
# set to max number of hyperspectral indices in experiment
bandLen = int(Bands[0])
print(bandLen)
bandOccurenceArr = np.zeros(bandLen).astype(int)
print('bandOccurenceArr: \n', bandOccurenceArr)
bandAbsSumArr = np.zeros(bandLen)
bandMaxArr = np.zeros(bandLen)
bandMinArr = np.zeros(bandLen)
bandAvgArr = np.zeros(bandLen)

indexArray = np.zeros(bandLen).astype(int)
for i in range(bandLen):
    indexArray[i] = indexArray[i] + i
print('indexArray: \n', indexArray, len(indexArray))


## Load SHAP values for each batch in experiment.  
### Aggregate results and rank features 

In [ ]:
first_time = True
index = 0
# Reload config
start = left = '[['
end = right = ']]'

for shap_vals_path in shap_files:
    
    index = index+1
    num_shap_files = len(shap_files)
    bandListStr = shap_vals_path[shap_vals_path.index(left)+len(left):shap_vals_path.index(right)]
    bandList = [e for e in bandListStr.split(',')]
    for bandNum in bandList:
        bandOccurenceArr[int(bandNum)-1] = (bandOccurenceArr[int(bandNum)-1]) + 1
    
    # Reload shap values
    shap_values0to50 = pickle.load(open(shap_vals_path, "rb"))
    num_bins = len(shap_values0to50)
    num_rows = len(shap_values0to50[0])
    num_bands = num_cols = len(shap_values0to50[0][0])
    if (index == 1): 
        print('\nProcessing shap file: index ', index,' out of ', num_shap_files, ':', os.path.basename(shap_vals_path))
        print('num_bins (', num_bins, ') num_rows (', num_rows, ') num_bands = num_cols(', num_bands, ') bandList(', bandList, ')')
        print('bandOccurenceArr after first mods(', bandOccurenceArr, ')')
    
    #  Walk across bins [0,4]
    for _bin in range(0, num_bins):
        if ((_bin == 0) and (index == 1)): print('len(shap_values0to50[',_bin,'])', len(shap_values0to50[_bin]))

        # get absolute value of all values in matrix
        shap_values0to50_bin_abs = np.abs(shap_values0to50[_bin])
        if ((_bin == 0) and (index == 1)): print('shap_values0to50_bin_abs.shape  : ', shap_values0to50_bin_abs.shape)
        
        # convert numpy matrix to dataframe and print sum of absolute values in each column
        df_shap_values0to50_bin_abs = pd.DataFrame(data=shap_values0to50_bin_abs)
        if ((_bin == 0) and (index == 1)): print('df_shap_values0to50_bin_abs.shape  : ', df_shap_values0to50_bin_abs.shape)
        
        # create a row (numpy array) where each column is the sum of the absolute values in that vertical column
        shap_values_abs_sum_axis0 = np.sum(df_shap_values0to50_bin_abs, axis=0)
        if ((_bin == 0) and (index == 1)): print('shap_values_abs_sum_axis0.shape  : ', shap_values_abs_sum_axis0.shape)

        # sort the row from low to high to determine the rank of the columns (e.g., features)
        shap_values_abs_sum_argsort = np.argsort(shap_values_abs_sum_axis0)
        if ((_bin == 0) and (index == 1)): print('L->H (index == 1):\n', shap_values_abs_sum_argsort)
        
        #  # increment Occurence value in the cell that contains the band number
        # if ((_bin == 0) and (index == 1)): print(bandOccurenceArr)
        # bandOccurenceArr[bandNum-1] = bandOccurenceArr[bandNum-1] + 1
        # if ((_bin == 0) and (index == 1)): print(bandOccurenceArr)
        
        # loop thru each column and update the corresponding statistics in the global row
        for _col in range(0, num_cols):
            bandNum = int(bandList[_col])
            if ((_bin == 0) and (index == 1)): print('bandNum[', bandNum,'] column[',_col,'] band[',bandList[_col].strip(),'] sum(abs)', df_shap_values0to50_bin_abs[_col].sum())
            
            # # increment Occurence value in the cell that contains the band number
            # if ((_bin == 0) and (index == 1)): print(bandOccurenceArr[bandNum-1])
            # bandOccurenceArr[bandNum-1] = bandOccurenceArr[bandNum-1] + 1
            # if ((_bin == 0) and (index == 1)): print(bandOccurenceArr[bandNum-1])
 
            # add the absolute value of the current shape_value to the existing value in the cell that is indexed by the band number
            # this allows us to keep a running total shap values per feature for downselection later....
            bandAbsSumArr[bandNum-1] = (bandAbsSumArr[bandNum-1] + df_shap_values0to50_bin_abs[_col].sum())
            
            # calculate average value of each band by dividing the absolute value by the number of occurences
#            bandAvgArr[bandNum-1] = (bandAbsSumArr[bandNum-1] / bandOccurenceArr[bandNum-1])


# bandAbsSumArr_argsort = np.argsort(bandAbsSumArr)
# print('Final L->H:\n', bandAbsSumArr_argsort)

# #df_bandAbsSumArr = pd.DataFrame(data=bandAbsSumArr)
# np_sum_df_bandAbsSumArr_axis0 = np.sum(bandAbsSumArr, axis=0) 
# print('np_sum_df_bandAbsSumArr_axis0.shape  : ', np_sum_df_bandAbsSumArr_axis0.shape)

# # sort the row from low to high to determine the rank of the columns (e.g., features)
# shap_values_abs_sum_argsort = np.argsort(np_sum_df_bandAbsSumArr_axis0)
# print('Final L->H:\n', shap_values_abs_sum_argsort)
 
with np.printoptions(precision=5, suppress=True):
    print('Final bandOccurenceArr (', bandOccurenceArr, ')')
    # print('Final bandAbsSumArr (',bandAbsSumArr)
    # print('Final L->H:\n', np.argsort(bandAbsSumArr))
#    print('bandAvgArr: ',bandAvgArr)
            
# #shap.summary_plot(shap_values0to50, X.iloc[0:50, :], plot_type="bar", feature_names=keyword)

In [ ]:

# print('\nDerive feature order based on average:')
# avg_shap_values_abs_sum_argsort_r = np.argsort(bandAvgArr)
# print('avg cumulative: \nmin()' , bandAvgArr.min(), '\nmax()', bandAvgArr.max(), '\n', bandAvgArr) 
# print('avg cumulative feature order L->H :' , avg_shap_values_abs_sum_argsort_r) 

print('\nDerive feature order based on aggregate sum:')
cum_shap_values_abs_sum_argsort_r = np.argsort(bandAbsSumArr)
print('raw cumulative: \nmin()' , bandAbsSumArr.min(), '\nmax()', bandAbsSumArr.max(), '\n', bandAbsSumArr) 
print('raw cumulative feature order L->H :' , cum_shap_values_abs_sum_argsort_r) 



In [ ]:
int(Bands[0])

In [ ]:
lowest_index = highest_index = -1
lowest = bandAbsSumArr[0] 
highest = bandAbsSumArr[0]
for gt in range(0, int(Bands[0])):
#        print(gt, bandAbsSumArr[gt], cum_shap_values_abs_sum_argsort_r[gt])
        if (cum_shap_values_abs_sum_argsort_r[gt] == bandAbsSumArr.min()):
                      print('index for bandAbsSumArr.min() is = ',  bandAbsSumArr[gt], ' at index=[', gt ,']')
        if (gt == 0):
                       print('lowest cumulative shap value is = ',  bandAbsSumArr[cum_shap_values_abs_sum_argsort_r[gt]], ' at index=[', cum_shap_values_abs_sum_argsort_r[gt] ,']')
        if (gt == (int(Bands[0]) - 1)):
                       print('highest cumulative shap value is = ',  bandAbsSumArr[cum_shap_values_abs_sum_argsort_r[gt]], ' at index=[', cum_shap_values_abs_sum_argsort_r[gt] ,']')
        if (bandAbsSumArr[gt] < lowest):
            lowest = bandAbsSumArr[gt]
            lowest_index = gt
        if (bandAbsSumArr[gt] > highest):
            highest = bandAbsSumArr[gt]
            highest_index = gt
print('lowest_index :', lowest_index,  'lowest value :', lowest)
print('highest_index :', highest_index,  'highest value :', highest)
print('raw cumulative: \nmin()' , bandAbsSumArr.min(), '\nmax()', bandAbsSumArr.max()) 


In [ ]:
band

In [ ]:
        # get band from band list

bandNum = int(bandList[i-1])
print(bandNum)

# print('i', str(i), ' shap_values_abs_sum_axis0[', str(bandNum-1),'][', str(i),']', str(shap_values_abs_sum_axis0[bandNum-1][i]), 
#        ' bands: ', str(cfg_obj.bandList), ' num_bands: ', str(num_bands), ' bandNum: ', str(bandNum))

# increment Occurence value in the cell that contains the band number
bandOccurenceArr[bandNum-1] = bandOccurenceArr[bandNum-1] + 1

# add the absolute value of the current shape_value to the existing value in the cell that is indexed by the band number
# this allows us to keep a running total shap values per feature for downselection later....
#        bandAbsSumArr[bandNum-1] = (bandAbsSumArr[bandNum-1] + shap_values_abs_sum_axis0[bandNum-1][i])
bandAbsSumArr[bandNum-1] = (bandAbsSumArr[bandNum-1] + shap_values_abs_sum_axis0[i][i])

occurences_axis0 = np.sum(shap_values_abs, axis=0)

# calculate average value of each band by dividing the absolute value by the number of occurences
bandAvgArr[bandNum-1] = (bandAbsSumArr[bandNum-1] / bandOccurenceArr[bandNum-1])


In [ ]:
#cfg = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_all21bands/01012024/bandListFile-save/CONFIG/Exp_all21bands::725617110916401361.keras[[20, 11, 13, 15, 21, 7, 9, 17, 1, 4]].cfg'
# cfg = cfg_files[0]
# cfg_obj = pickle.load(open(cfg, "rb"))
# print(cfg_obj)
# shap_vals_path = cfg_obj.shap_path
# print('shap_vals_path:', shap_vals_path)

In [ ]:
# shap_vals_path = cfg_obj.shap_path
# shap_values0to50 = pickle.load(open(shap_vals_path, "rb"))
# print(len(shap_values0to50[0][0]), cfg_obj.bandList)

In [ ]:
first_time = True
# Reload config
for file in cfg_files:
    print('Processing file: \n', file)
    cfg_obj = pickle.load(open(file, "rb"))   
    cfg_path = cfg_obj.configFile
#    num_bands = len(cfg_obj.bandList)
    shap_vals_path = cfg_obj.shap_path
    modelPath = cfg_obj.model_path
    # archive_test_generator_id = cfg_obj.trial_path
    keyword = cfg_obj.bandList

#     # Reload test data
#     test_generator = pickle.load(open(archive_test_generator_id, "rb"))

#     # Transpose test data for shap_values generation
#     import pandas as pd
#     df = pd.DataFrame(data=test_generator.file_x_stack)
#     X = df.transpose()
#     X.iloc[:, :]

    # Reload shap values
    shap_values0to50 = pickle.load(open(shap_vals_path, "rb"))
    num_bands = len(shap_values0to50[0][0])
                    
    shap_values_abs = np.abs(shap_values0to50)

    shap_values_abs_sum_axis0 = np.sum(shap_values_abs, axis=0)

    shap_values_abs_sum_argsort = np.argsort(shap_values_abs_sum_axis0)
    
    shap_values_abs_sum_argsort_r = np.argsort(-shap_values_abs_sum_axis0)
    
    print("\n**** feature_order = np.argsort(np.sum(np.abs(shap_values), axis=0))")
    print('shap_vals_path: ', str(shap_vals_path))
    print('bands: ', str(cfg_obj.bandList))
    print('sum  : ', shap_values_abs_sum_axis0[0])
    print('L->H :', shap_values_abs_sum_argsort[0])
    print('H->L :', shap_values_abs_sum_argsort_r[0])
    
    for i in range(num_bands):

        # get band from band list
        bandNum = cfg_obj.bandList[i]
        
        # print('i', str(i), ' shap_values_abs_sum_axis0[', str(bandNum-1),'][', str(i),']', str(shap_values_abs_sum_axis0[bandNum-1][i]), 
        #       ' bands: ', str(cfg_obj.bandList), ' num_bands: ', str(num_bands), ' bandNum: ', str(bandNum))
        
        # increment Occurence value in the cell that contains the band number
        bandOccurenceArr[bandNum-1] = bandOccurenceArr[bandNum-1] + 1
    
        # add the absolute value of the current shape_value to the existing value in the cell that is indexed by the band number
        # this allows us to keep a running total shap values per feature for downselection later....
        bandAbsSumArr[bandNum-1] = (bandAbsSumArr[bandNum-1] + shap_values_abs_sum_axis0[bandNum-1][i])
    
        occurences_axis0 = np.sum(shap_values_abs, axis=0)
    
        # calculate average value of each band by dividing the absolute value by the number of occurences
        bandAvgArr[bandNum-1] = (bandAbsSumArr[bandNum-1] / bandOccurenceArr[bandNum-1])

    
    with np.printoptions(precision=5, suppress=True):
        print(indexArray)
        print(bandOccurenceArr)
        print(bandAbsSumArr)
        print(bandAvgArr)
        
#shap.summary_plot(shap_values0to50, X.iloc[0:50, :], plot_type="bar", feature_names=keyword)

# ***Derive aggregated feature order***

**================================================= RANK ==================================**

In [ ]:

print('\nDerive feature order based on average:')
avg_shap_values_abs_sum_argsort_r = np.argsort(bandAvgArr)
print('avg cumulative: \nmin()' , bandAvgArr.min(), '\nmax()', bandAvgArr.max(), '\n', bandAvgArr) 
print('avg cumulative feature order L->H :' , avg_shap_values_abs_sum_argsort_r) 

print('\nDerive feature order based on aggregate sum:')
cum_shap_values_abs_sum_argsort_r = np.argsort(bandAbsSumArr)
print('raw cumulative: \nmin()' , bandAbsSumArr.min(), '\nmax()', bandAbsSumArr.max(), '\n', bandAbsSumArr) 
print('raw cumulative feature order L->H :' , cum_shap_values_abs_sum_argsort_r) 



**================================================= RANK ==================================**

# *FYI...*

## Calculate additional SHAP values (sum, abs, sort)

In [ ]:
# from _beeswarm.py.   
#        feature_order = np.argsort(np.sum(np.abs(shap_values), axis=0))


def permutation_importance(shap_values):
#    print("\nshap_values:\n", shap_values.shape)
#    print("\nshap_values:\n", shap_values, shap_values.dtype, shap_values.shape)

    shap_values_sum_axis1 = np.sum(shap_values, axis=1)
#    print("\nshap_values_sum_axis1: np.sum(shap_values_sum, axis=1) [sum across row]\n", shap_values_sum_axis1)

    shap_values_sum_axis0 = np.sum(shap_values, axis=0)
#    print("\nshap_values_sum_axis0 : np.sum(shap_values, axis=0) [sum down column]\n", shap_values_sum_axis0)

    shap_values_sum_argsort = np.argsort(shap_values_sum_axis0)
#    print ("\nshap_values_sum_argsort: np.argsort(shap_values_sum_axis0)  [sum down column]\n", shap_values_sum_argsort)

    shap_values_abs = np.abs(shap_values)
#    print("\nshap_values_abs: (np.abs(nd_a))\n", shap_values_abs)

    shap_values_abs_sum_axis1 = np.sum(shap_values_abs, axis=1)
#    print("\nshap_values_abs_sum_axis1: np.sum(shap_values_abs, axis=1) [sum across row]\n", shap_values_abs_sum_axis1)

    shap_values_abs_sum_axis0 = np.sum(shap_values_abs, axis=0)
#    print("\nshap_values_abs_sum_axis0: np.sum(shap_values_abs, axis=0) [sum down column]\n", shap_values_abs_sum_axis0)

    shap_values_abs_sum_argsort = np.argsort(shap_values_abs_sum_axis0)
#    print("\n**** feature_order = np.argsort(np.sum(np.abs(shap_values), axis=0))")
#    print ("shap_values_abs_sum_argsort: np.argsort(shap_values_abs_sum_axis0)  [sum down column]\n *", shap_values_abs_sum_argsort)

    feature_order = shap_values_abs_sum_argsort
#    if not (shap_values_abs_sum_argsort[0] == feature_order[0]): raise AssertionError
#    print('\nfeature_order:\n', feature_order)
    
    return shap_values_sum_axis1, shap_values_sum_axis0, shap_values_sum_argsort, \
        shap_values_abs, shap_values_abs_sum_axis1, shap_values_abs_sum_axis0, shap_values_abs_sum_argsort, feature_order

In [ ]:
# example row/column statistics with test matrix that is easy to predict.  Note that argsort goes from low to high.
a = [[1, 2, 3, 4, 5, 6, -7], [1, 2, 3, 4, 5, -29, 7], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7], [-1, 2, 3, 4, 5, 6, 7]]
print(np.array(a))
permutation_importance(np.array(a))
    # return shap_values_sum_axis1, shap_values_sum_axis0, shap_values_sum_argsort, \
    #     shap_values_abs, shap_values_abs_sum_axis1, shap_values_abs_sum_axis0, shap_values_abs_sum_argsort, feature_order

In [ ]:
print(i, bandNum, bandAbsSumArr[bandNum-1], shap_values_abs_sum_axis0[bandNum-1][i])
print('sum  : ', shap_values_abs_sum_axis0[0][1])


## Interrogate configuration object (optional)

In [ ]:
# Reload config
cfg_obj = pickle.load(open(cfg_files[0], "rb"))   
print(cfg_obj)
print(cfg_obj.bandList)
#mpfConfig = (MpfConfig) cfg_obj
#print(mpfConfig)

In [ ]:
print(cfg_obj.model_config)
print(cfg_obj.model_config['model_name'])

In [ ]:
print(cfg_obj.mlflow_config)
print(cfg_obj.mlflow_config['EXPERIMENT_NAME'])

In [ ]:
print(cfg_obj.X)
print(cfg_obj.data_generator_config)
print(cfg_obj.data_generator_config['data_path'])
print(cfg_obj.model_path)

In [ ]:
cfg_path = cfg_obj.configFile
num_bands = len(cfg_obj.bandList)
shap_vals_path = cfg_obj.shap_path
modelPath = cfg_obj.model_path
archive_test_generator_id = cfg_obj.trial_path
keyword = cfg_obj.bandList

print(cfg_path)
print(modelPath)
print(archive_test_generator_id)
print(shap_vals_path)
print(cfg_obj.bandList, num_bands)

## Load test data to run additional plots (optional)

In [ ]:
# Reload test data
test_generator = pickle.load(open(archive_test_generator_id, "rb"))

# Transpose test data for shap_values generation
import pandas as pd
df = pd.DataFrame(data=test_generator.file_x_stack)
X = df.transpose()
X.iloc[:, :]
print(X.shape, X.shape[0], X.shape[1])

In [ ]:
# Reload shap values
shap_values0to50 = pickle.load(open(shap_vals_path, "rb"))

In [ ]:
# print according to most in particular bin:  0 < -1.0, 1 < -0.5, 2 < 0, 3 < 0.5, 4 < 1.0
print('bin value 0: [-1.0, -0.5]] = summary_plot(shap_values0to50[0]') 
shap.summary_plot(shap_values0to50[4], X.iloc[0:50, :], plot_type="bar", feature_names=keyword)

In [ ]:
shap.summary_plot(shap_values0to50, X, plot_type="bar", feature_names=keyword)

In [ ]:
shap_values_np = np.array(shap_values0to50)
shap_values_np[0].shape

In [ ]:
df = pd.DataFrame(data=shap_values0to50[4])
df.head(5)

In [ ]:
# example row/column statistics with test matrix that is easy to predict.  Note that argsort goes from low to high.
#print(np.array(shap_values50))
_shap_values50_sum_axis1, _shap_values50_sum_axis0, _shap_value50s_sum_argsort, \
        _shap_values50_abs, _shap_values50_abs_sum_axis1, _shap_values50_abs_sum_axis0, _shap_values50_abs_sum_argsort, _feature50_order = \
        permutation_importance(np.array(shap_values0to50[0]))

print ('shap_values0to50[0] feature_order:\n', _feature50_order.shape, '\n', _feature50_order)

_shap_values50_sum_axis1, _shap_values50_sum_axis0, _shap_value50s_sum_argsort, \
        _shap_values50_abs, _shap_values50_abs_sum_axis1, _shap_values50_abs_sum_axis0, _shap_values50_abs_sum_argsort, _feature50_order = \
        permutation_importance(np.array(shap_values0to50))

print ('\nshap_values0to50[0] feature_order:\n', _feature50_order[0].shape, '\n', _feature50_order[0], '\n',  _shap_values50_abs_sum_axis0[0])
print ('\nshap_values0to50 feature_order:\n', _feature50_order.shape, '\n', _feature50_order, '\n',  _shap_values50_abs_sum_axis0)

In [ ]:
# example row/column statistics with test matrix that is easy to predict.  Note that argsort goes from low to high.
#print(np.array(shap_values0to50))
permutation_importance(np.array(shap_values0to50))

In [ ]:
#!/usr/bin/env python3
from functools import partial
from itertools import repeat
from multiprocessing import Pool, freeze_support

def func(a, b):
    print(a+b)
    return a + b

def main():
    a_args = [1,2,3]
    second_arg = 1
    with Pool() as pool:
        L = pool.starmap(func, [(1, 1), (2, 1), (3, 1)])
        M = pool.starmap(func, zip(a_args, repeat(second_arg)))
        N = pool.map(partial(func, b=second_arg), a_args)
        assert L == M == N

if __name__=="__main__":
    freeze_support()
    main()

In [ ]:
import multiprocessing

pool = multiprocessing.Pool(processes=6)
pool.starmap(func, [(1, 1), (2, 1), (3, 1)])
pool.close()
pool.join()